# gzip - Read and Write GNU zip Files

Purpose:	Read and write gzip files.
    
The gzip module provides a file-like interface to GNU zip files, using zlib to compress and uncompress the data.

## Writing Compressed Files

The module-level function open() creates an instance of the file-like class GzipFile. The usual methods for writing and reading bytes are provided.

In [1]:
!python3 gzip_write.py

example.txt.gz contains 75 bytes
application/x-gzip; charset=binary


To write data into a compressed file, open the file with mode 'wb'. This example wraps the GzipFile with a TextIOWrapper from the io module to encode Unicode text to bytes suitable for compression.

Different amounts of compression can be used by passing a compresslevel argument. Valid values range from 0 to 9, inclusive. Lower values are faster and result in less compression. Higher values are slower and compress more, up to a point.

In [2]:
# gzip_compresslevel.py
import gzip
import io
import os
import hashlib


def get_hash(data):
    return hashlib.md5(data).hexdigest()


data = open('example.py', 'r').read() * 1024
cksum = get_hash(data.encode('utf-8'))


print('Level  Size        Checksum')
print('-----  ----------  ---------------------------------')
print('data   {:>10}  {}'.format(len(data), cksum))

for i in range(0, 10):
    filename = 'compress-level-{}.gz'.format(i)
    with gzip.open(filename, 'wb', compresslevel=i) as output:
        with io.TextIOWrapper(output, encoding='utf-8') as enc:
            enc.write(data)
    size = os.stat(filename).st_size
    cksum = get_hash(open(filename, 'rb').read())
    print('{:>5d}  {:>10d}  {}'.format(i, size, cksum))

Level  Size        Checksum
-----  ----------  ---------------------------------
data        95232  3d4fd702cdce150f62a3539c10f42fdc
    0       95287  5520efb889ef04a3ab66eadfd41dbc4f
    1         781  41f80b37b1187ba272dff5a619510a06
    2         781  ba4075a0068ce954c5600ac70b12715e
    3         781  5c8fff92aabe1434531da21eeb95267e
    4         467  0264e1a50ea8bacc28a48ddcf7f5b659
    5         467  941abc7f4c11bc95b3338559806ec0e8
    6         467  841d39e31564f96bd588dcdd82015471
    7         467  ba8b1b1b60cbdb7f6c9a088f76dfb06d
    8         467  79ee9bffde81bd37845ce7cd2c4e1562
    9         467  2829ddd2383fcb798db693c2b0615d02


A GzipFile instance also includes a writelines() method that can be used to write a sequence of strings.

In [3]:
! python3 gzip_writelines.py

The same line, over and over.
The same line, over and over.
The same line, over and over.
The same line, over and over.
The same line, over and over.
The same line, over and over.
The same line, over and over.
The same line, over and over.
The same line, over and over.
The same line, over and over.


## Reading Compressed Data

To read data back from previously compressed files, open the file with binary read mode ('rb') so no text-based translation of line endings or Unicode decoding is performed.

In [4]:
# gzip_read.py
import gzip
import io

with gzip.open('example.txt.gz', 'rb') as input_file:
    with io.TextIOWrapper(input_file, encoding='utf-8') as dec:
        print(dec.read())

Contents of the example file go here.



This example reads the file written by gzip_write.py from the previous section, using a TextIOWrapper to decode the text after it is decompressed.

While reading a file, it is also possible to seek and read only part of the data.

The seek() position is relative to the uncompressed data, so the caller does not need to know that the data file is compressed.

In [5]:
# gzip_seek.py
import gzip

with gzip.open('example.txt.gz', 'rb') as input_file:
    print('Entire file:')
    all_data = input_file.read()
    print(all_data)

    expected = all_data[5:15]

    # rewind to beginning
    input_file.seek(0)

    # move ahead 5 bytes
    input_file.seek(5)
    print('Starting at position 5 for 10 bytes:')
    partial = input_file.read(10)
    print(partial)

    print()
    print(expected == partial)

Entire file:
b'Contents of the example file go here.\n'
Starting at position 5 for 10 bytes:
b'nts of the'

True


## Working with Streams

The GzipFile class can be used to wrap other types of data streams so they can use compression as well. This is useful when the data is being transmitted over a socket or an existing (already open) file handle. A BytesIO buffer can also be used.

One benefit of using GzipFile over zlib is that it supports the file API. However, when re-reading the previously compressed data, an explicit length is passed to read(). Leaving the length off resulted in a CRC error, possibly because BytesIO returned an empty string before reporting EOF. When working with streams of compressed data, either prefix the data with an integer representing the actual amount of data to be read or use the incremental decompression API in zlib.

In [6]:
# gzip_BytesIO.py
import gzip
from io import BytesIO
import binascii

uncompressed_data = b'The same line, over and over.\n' * 10
print('UNCOMPRESSED:', len(uncompressed_data))
print(uncompressed_data)

buf = BytesIO()
with gzip.GzipFile(mode='wb', fileobj=buf) as f:
    f.write(uncompressed_data)

compressed_data = buf.getvalue()
print('\nCOMPRESSED:', len(compressed_data))
print(binascii.hexlify(compressed_data))

inbuffer = BytesIO(compressed_data)
with gzip.GzipFile(mode='rb', fileobj=inbuffer) as f:
    reread_data = f.read(len(uncompressed_data))

print('\nREREAD:', len(reread_data))
print(reread_data)

UNCOMPRESSED: 300
b'The same line, over and over.\nThe same line, over and over.\nThe same line, over and over.\nThe same line, over and over.\nThe same line, over and over.\nThe same line, over and over.\nThe same line, over and over.\nThe same line, over and over.\nThe same line, over and over.\nThe same line, over and over.\n'

COMPRESSED: 51
b'1f8b0800aa711a5902ff0bc94855284ecc4d55c8c9cc4bd551c82f4b2d5248cc4b0133f4b8424665916401d3e717802c010000'

REREAD: 300
b'The same line, over and over.\nThe same line, over and over.\nThe same line, over and over.\nThe same line, over and over.\nThe same line, over and over.\nThe same line, over and over.\nThe same line, over and over.\nThe same line, over and over.\nThe same line, over and over.\nThe same line, over and over.\n'


In [7]:
# cleanup the test files
!rm compress-level*.gz example.txt.gz example_lines.txt.gz